In [22]:
import pandas as pd
import numpy as np

In [23]:
def unstack_data(df):
    rows = []
    rfids = [i for i in df.rfid.unique()]

    for rfid in rfids:
        temp = df[df.rfid==rfid]
        temp = temp.reset_index(drop=True)
        temp.sort_values(['trial_id'],inplace=True)
        temp.set_index('trial_id',inplace=True)
        temp_row = temp.iloc[:,1:].unstack().to_frame().sort_index(level=1).T
        temp_row.insert(loc=0, column='rfid', value=temp.rfid.unique())
        temp_row = temp_row.reset_index(drop=True)
        rows.append(temp_row)
        
    return rows


def deserialize_data(ts):
    if pd.isnull(ts):
        return []
    return [float(i) for i in ts.split()]

## subject

In [24]:
# the "index" table, all other tables should left join to it

In [25]:
sub = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/datastream_subject_table.csv')
#sub = sub[[i for i in sub.columns.tolist() if 'date' not in i]]
print(sub.shape)
sub.head()

(2381, 105)


,rfid,rat,cohort,experiment_group,drug_group,sex,arrival_date,age_at_arrival,uv,brevital,...,age_at_surgery,date_of_death,days_of_experiment,exit_day,last_good_session,exit_code,complete,tissue_collected,exit_notes,replaced_by
0,933000120124701,F1,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000120124703,F2,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000120117342,F3,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000120117333,F4,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000120117347,F5,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,2017-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
sub.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/subject.csv')

## measurement

In [28]:
mmt = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/datastream_measurement_table.csv')
mmt.head()

,rfid,measurement_name,measurement_value,drug_group,cohort,measure_number,date_measured,technician
0,933000120124701,urine,NaN,cocaine,1,1,NaN,NaN
1,933000120124701,urine,NaN,cocaine,1,2,NaN,NaN
2,933000120124701,weight,177.0,cocaine,1,1,2017-07-24,NaN
3,933000120124701,weight,179.0,cocaine,1,2,2017-08-04,NaN
4,933000120124701,weight,190.0,cocaine,1,3,2017-08-11,NaN


In [29]:
mmt['measure_id'] = mmt['measurement_name'] + ['_']*len(mmt) + mmt['measure_number'].astype(str)
mmt_trimmed = mmt.drop(columns=['measurement_name','drug_group','cohort','measure_number','date_measured'])
mmt_trimmed.head()

,rfid,measurement_value,technician,measure_id
0,933000120124701,NaN,NaN,urine_1
1,933000120124701,NaN,NaN,urine_2
2,933000120124701,177.0,NaN,weight_1
3,933000120124701,179.0,NaN,weight_2
4,933000120124701,190.0,NaN,weight_3


In [30]:
rows = []
rfids = [i for i in mmt.rfid.unique()]

for rfid in rfids:
    temp = mmt_trimmed[mmt_trimmed.rfid == rfid]
    temp = temp.reset_index(drop=True)
    temp.sort_values(['measure_id'],inplace=True)
    temp.set_index('measure_id',inplace=True)
    temp_row = temp.iloc[:,1:].unstack().to_frame().sort_index(level=1).T
    temp_row.insert(loc=0, column='rfid', value=temp.rfid.unique())
    temp_row = temp_row.reset_index(drop=True)
    rows.append(temp_row)

In [31]:
mmt_unstacked = pd.concat(rows,ignore_index=True)
mmt_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in mmt_unstacked.columns[1:]]
print(mmt_unstacked.shape)
mmt_unstacked.head()

(2381, 37)


,rfid,measurement_value_feces_1,technician_feces_1,measurement_value_feces_2,technician_feces_2,measurement_value_feces_3,technician_feces_3,measurement_value_feces_4,technician_feces_4,measurement_value_urine_1,...,measurement_value_weight_7,technician_weight_7,measurement_value_weight_8,technician_weight_8,measurement_value_weight_9,technician_weight_9,measurement_value_weight_11,technician_weight_11,measurement_value_weight_12,technician_weight_12
0,933000120124701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000120124703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000120117342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000120117333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000120117347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
mmt_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/measurement_unstacked.csv')

## irritability

In [33]:
irr = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/irritability.csv')
irr.drop('#',axis=1,inplace=True)
irr.replace(-999.0,np.nan, inplace=True)
irr.head()

,subject,rfid,cohort,sex,group,def_bsl_irr_1,agg_bsl_irr_1,irr_bsl_tot_1,irr_bsl_scorer_1,def_bsl_irr_2,...,def_drug_irr_3,agg_drug_irr_3,irr_drug_total_3,irr_drug_scorer_3,def_drug_ave,agg_drug_ave,total_drug_ave,diff_ave_def,diff_ave_agg,diff_ave_total
0,F1,933000120124701,1.0,F,cocaine,NaN,NaN,NaN,NaN,NaN,...,15.0,11.0,26.0,adam,13.333333,7.333333,20.666667,NaN,NaN,NaN
1,F2,933000120124703,1.0,F,cocaine,NaN,NaN,NaN,NaN,NaN,...,16.0,4.0,20.0,adam,14.666667,3.000000,17.666667,NaN,NaN,NaN
2,F3,933000120117342,1.0,F,cocaine,NaN,NaN,NaN,NaN,NaN,...,9.0,15.0,24.0,adam,8.666667,12.666667,21.333333,NaN,NaN,NaN
3,F4,933000120117333,1.0,F,cocaine,NaN,NaN,NaN,NaN,NaN,...,2.0,3.0,5.0,adam,2.000000,2.333333,4.333333,NaN,NaN,NaN
4,F5,933000120117347,1.0,F,cocaine,NaN,NaN,NaN,NaN,NaN,...,6.0,0.0,6.0,adam,9.333333,0.000000,9.333333,NaN,NaN,NaN


In [35]:
irr_trimmed = irr.drop(columns=['subject', 'cohort', 'sex', 'group'])
print(irr_trimmed.shape)
irr_trimmed.head()

(662, 34)


,rfid,def_bsl_irr_1,agg_bsl_irr_1,irr_bsl_tot_1,irr_bsl_scorer_1,def_bsl_irr_2,agg_bsl_irr_2,irr_bsl_total_2,irr_bsl_scorer_2,def_bsl_irr_3,...,def_drug_irr_3,agg_drug_irr_3,irr_drug_total_3,irr_drug_scorer_3,def_drug_ave,agg_drug_ave,total_drug_ave,diff_ave_def,diff_ave_agg,diff_ave_total
0,933000120124701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,11.0,26.0,adam,13.333333,7.333333,20.666667,NaN,NaN,NaN
1,933000120124703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,16.0,4.0,20.0,adam,14.666667,3.000000,17.666667,NaN,NaN,NaN
2,933000120117342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.0,15.0,24.0,adam,8.666667,12.666667,21.333333,NaN,NaN,NaN
3,933000120117333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.0,3.0,5.0,adam,2.000000,2.333333,4.333333,NaN,NaN,NaN
4,933000120117347,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.0,0.0,6.0,adam,9.333333,0.000000,9.333333,NaN,NaN,NaN


In [37]:
irr_trimmed.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/irritability.csv')

## tail immersion

In [38]:
ti = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/tail_immersion.csv')
ti.drop('#',axis=1,inplace=True)
ti.head()

,rfid,subject,cohort,sex,experiment_group,drug,tail_immersion_1_time,tail_immersion_2_time,tail_immersion_3_time,tail_immersion_difference_tolerance,tail_immersion_1_date,tail_immersion_2_date,tail_immersion_3_date
0,933000320045738,F301,3,Female,Drug,oxycodone,4.0,5.34,8.10,-2.76,2019-01-07,2019-01-21,2019-02-15
1,933000320045726,F302,3,Female,Drug,oxycodone,4.0,5.94,7.00,-1.06,2019-01-07,2019-01-21,2019-02-15
2,933000320045747,F303,3,Female,Drug,oxycodone,4.0,5.53,6.31,-0.78,2019-01-07,2019-01-21,2019-02-15
3,933000320045734,F304,3,Female,Drug,oxycodone,2.0,5.19,6.85,-1.66,2019-01-07,2019-01-21,2019-02-15
4,933000320045669,F305,3,Female,Drug,oxycodone,3.0,5.25,5.59,-0.34,2019-01-07,2019-01-21,2019-02-15


In [40]:
ti_trimmed = ti.drop(columns=['subject', 'cohort', 'sex', 'drug','tail_immersion_1_date',
                              'tail_immersion_2_date','tail_immersion_3_date','experiment_group'])
print(ti_trimmed.shape)
ti_trimmed.head()

(976, 5)


,rfid,tail_immersion_1_time,tail_immersion_2_time,tail_immersion_3_time,tail_immersion_difference_tolerance
0,933000320045738,4.0,5.34,8.10,-2.76
1,933000320045726,4.0,5.94,7.00,-1.06
2,933000320045747,4.0,5.53,6.31,-0.78
3,933000320045734,2.0,5.19,6.85,-1.66
4,933000320045669,3.0,5.25,5.59,-0.34


In [41]:
ti_trimmed.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/tailimmersion.csv')

## voy frey

In [42]:
vf = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/von_frey.csv')
vf.drop('#',axis=1,inplace=True)
vf.head()

,rfid,subject,cohort,sex,experiment_group,drug,vf1_right_force_1,vf1_right_force_2,vf1_right_force_3,vf1_right_force_avg,...,vf2_left_time_1,vf2_left_time_2,vf2_left_time_3,vf2_left_avg,von_frey_2_date,von_frey_1_force,von_frey_1_time,von_frey_2_force,von_frey_2_time,von_frey_difference_force
0,933000320186887,M773,7,Male,Drug,oxycodone,31.9,8.0,26.9,22.3,...,NaN,NaN,NaN,NaN,2020-04-26,22.15,10.95,NaN,NaN,NaN
1,933000120138681,F101,1,Female,Drug,oxycodone,13.7,13.3,27.7,18.2,...,12.8,11.0,11.1,11.6,2018-08-30,23.40,13.70,24.55,12.20,-1.15
2,933000120138443,F102,1,Female,Drug,oxycodone,16.1,5.4,15.3,12.3,...,4.7,8.4,7.6,6.9,2018-08-30,13.65,16.00,13.05,6.40,0.60
3,933000120138699,F103,1,Female,Drug,oxycodone,27.9,24.7,12.9,21.8,...,6.8,3.9,2.8,4.5,2018-08-30,18.15,17.10,11.90,5.80,6.25
4,933000120138703,F104,1,Female,Drug,oxycodone,3.8,10.1,5.0,6.3,...,8.7,15.9,8.7,11.1,2018-08-30,8.55,12.35,19.90,9.85,-11.35


In [43]:
vf_trimmed = vf.drop(columns=['subject', 'cohort', 'sex', 'drug',
                              'von_frey_1_date', 'von_frey_2_date','experiment_group'])
print(vf_trimmed.shape)
vf_trimmed.head()

(1066, 38)


,rfid,vf1_right_force_1,vf1_right_force_2,vf1_right_force_3,vf1_right_force_avg,vf1_right_time_1,vf1_right_time_2,vf1_right_time_3,vf1_right_avg,vf1_left_force_1,...,vf2_left_force_avg,vf2_left_time_1,vf2_left_time_2,vf2_left_time_3,vf2_left_avg,von_frey_1_force,von_frey_1_time,von_frey_2_force,von_frey_2_time,von_frey_difference_force
0,933000320186887,31.9,8.0,26.9,22.3,15.9,3.8,13.4,11.0,15.5,...,NaN,NaN,NaN,NaN,NaN,22.15,10.95,NaN,NaN,NaN
1,933000120138681,13.7,13.3,27.7,18.2,6.7,6.5,13.8,9.0,30.3,...,23.4,12.8,11.0,11.1,11.6,23.40,13.70,24.55,12.20,-1.15
2,933000120138443,16.1,5.4,15.3,12.3,7.9,2.5,7.5,6.0,16.1,...,14.1,4.7,8.4,7.6,6.9,13.65,16.00,13.05,6.40,0.60
3,933000120138699,27.9,24.7,12.9,21.8,13.9,12.3,6.3,10.8,17.2,...,9.4,6.8,3.9,2.8,4.5,18.15,17.10,11.90,5.80,6.25
4,933000120138703,3.8,10.1,5.0,6.3,1.7,4.9,2.3,3.0,10.1,...,22.4,8.7,15.9,8.7,11.1,8.55,12.35,19.90,9.85,-11.35


In [45]:
vf_trimmed.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/vonfrey_unstacked.csv')

## trial_note

In [46]:
note = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_note.csv')
note.drop('#',axis=1,inplace=True)
note.head()

,rfid,subject,cohort,sex,drug,experiment_group,trial_id,start_date,code,to_do,note
0,933000320047006,M987,9,Male,Cocaine,Drug,LGA01,2019-10-08,Disconnect,Keep,NaN
1,933000320186832,M1072,10,Male,Cocaine,Drug,LGA02,2020-01-14,Died,Discard,NaN
2,933000320188172,M1565,15,Male,Cocaine,Drug,LGA01,2021-01-13,Tangled,Keep,NaN
3,933000320188113,M1553,15,Male,Cocaine,Drug,LGA06,2021-01-21,Tangled,Discard,NaN
4,933000320187895,M1572,15,Male,Cocaine,Drug,LGA11,2021-01-28,Sick/wound,Keep,NaN


In [47]:
note['drug'] = note['drug'].str.lower()
note['to_do'] = note['to_do'].apply(lambda x: x.lower().strip())

In [48]:
note_filter = note[note['to_do']=='discard'][['rfid','trial_id','drug','to_do']]
note_filter.reset_index(drop=True)

,rfid,trial_id,drug,to_do
0,933000320186832,LGA02,cocaine,discard
1,933000320188113,LGA06,cocaine,discard
2,933000320188614,LGA05,cocaine,discard
3,933000320499966,LGA06,cocaine,discard
4,933000320046139,LGA06,oxycodone,discard
...,...,...,...,...
1016,933000320697653,TREATMENT03,oxycodone,discard
1017,933000320697556,LGA18,oxycodone,discard
1018,933000320697558,LGA18,oxycodone,discard
1019,933000320697654,LGA18,oxycodone,discard


In [49]:
note_filter.replace('SHOCK','SHOCK_V3',inplace=True)
note_filter.replace('PRE-SHOCK','PRESHOCK',inplace=True)
note_filter.replace('TREAMENT02','TREATMENT02',inplace=True)
note_filter.replace('TREAMENT03','TREATMENT03',inplace=True)
note_filter.replace('PRE-SHOCK','PRESHOCK',inplace=True)

## trial_shock

In [50]:
shock = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_shock.csv')
shock.head()

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000320046919,F809,BSB273B,8,SHOCK_V3,cocaine,1,10:23:32,11:25:09,2019-08-02,2019-08-02,23.0,0.0,6.0,20.0,12.0,8 9 10 13 15 18,15 53 100 433 588 760 972 1152 1250 1498 1615 ...
1,933000320047277,M863,BSB273B,8,SHOCK_V3,cocaine,13,10:24:23,11:25:09,2019-08-02,2019-08-02,15.0,31.0,4.0,15.0,11.0,4 5 9 11,30 714 804 927 1094 1229 1389 1935 2098 2350 2...
2,933000320047132,M867,BSB273B,8,SHOCK_V3,cocaine,9,12:05:13,13:05:55,2019-08-02,2019-08-02,10.0,4.0,3.0,10.0,4.0,6 8 9,25 148 257 360 399 480 747 1555 1638 3020
3,933000320047119,F829,BSB273B,8,SHOCK_V3,cocaine,5,13:29:23,14:30:00,2019-08-02,2019-08-02,43.0,1.0,8.0,27.0,26.0,1 7 10 16 17 19 23 27,13 63 85 126 168 193 218 243 774 820 953 1080 ...
4,933000320047312,F805,BSB273B,8,SHOCK_V3,cocaine,5,09:01:23,10:02:06,2019-08-02,2019-08-02,52.0,0.0,12.0,44.0,39.0,5 7 9 11 18 20 24 28 29 35 39 40,140 193 230 261 309 420 593 804 924 991 1097 1...


In [51]:
shock['rewards_got_shock'] = shock['rewards_got_shock'].apply(deserialize_data)
shock['reward_timestamps'] = shock['reward_timestamps'].apply(deserialize_data)

In [52]:
shock_trimmed = shock.drop(columns=['subject', 'room', 'cohort', 'box','start_date','end_date'])
print(shock_trimmed.shape)
shock_trimmed.head()

(2027, 12)


,rfid,trial_id,drug,start_time,end_time,total_active_lever_presses,total_inactive_lever_presses,total_shocks,total_reward,rewards_after_first_shock,rewards_got_shock,reward_timestamps
0,933000320046919,SHOCK_V3,cocaine,10:23:32,11:25:09,23.0,0.0,6.0,20.0,12.0,"[8.0, 9.0, 10.0, 13.0, 15.0, 18.0]","[15.0, 53.0, 100.0, 433.0, 588.0, 760.0, 972.0..."
1,933000320047277,SHOCK_V3,cocaine,10:24:23,11:25:09,15.0,31.0,4.0,15.0,11.0,"[4.0, 5.0, 9.0, 11.0]","[30.0, 714.0, 804.0, 927.0, 1094.0, 1229.0, 13..."
2,933000320047132,SHOCK_V3,cocaine,12:05:13,13:05:55,10.0,4.0,3.0,10.0,4.0,"[6.0, 8.0, 9.0]","[25.0, 148.0, 257.0, 360.0, 399.0, 480.0, 747...."
3,933000320047119,SHOCK_V3,cocaine,13:29:23,14:30:00,43.0,1.0,8.0,27.0,26.0,"[1.0, 7.0, 10.0, 16.0, 17.0, 19.0, 23.0, 27.0]","[13.0, 63.0, 85.0, 126.0, 168.0, 193.0, 218.0,..."
4,933000320047312,SHOCK_V3,cocaine,09:01:23,10:02:06,52.0,0.0,12.0,44.0,39.0,"[5.0, 7.0, 9.0, 11.0, 18.0, 20.0, 24.0, 28.0, ...","[140.0, 193.0, 230.0, 261.0, 309.0, 420.0, 593..."


In [53]:
shock_keep = pd.merge(shock_trimmed, note_filter, how='left', left_on=['rfid','trial_id','drug'], right_on=['rfid','trial_id','drug'])
shock_keep = shock_keep[~(shock_keep['to_do']=='discard')]
shock_keep.drop(columns=['drug','to_do'],inplace=True)
shock_keep.shape

(2021, 11)

In [54]:
rows = unstack_data(shock_keep)
shock_unstacked = pd.concat(rows,ignore_index=True)
shock_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in shock_unstacked.columns[1:]]
print(shock_unstacked.shape)
shock_unstacked.head()

(891, 37)


,rfid,end_time_SHOCK_V3,reward_timestamps_SHOCK_V3,rewards_after_first_shock_SHOCK_V3,rewards_got_shock_SHOCK_V3,start_time_SHOCK_V3,total_active_lever_presses_SHOCK_V3,total_inactive_lever_presses_SHOCK_V3,total_reward_SHOCK_V3,total_shocks_SHOCK_V3,...,total_shocks_SHOCK_V1,end_time_SHOCK_V2,reward_timestamps_SHOCK_V2,rewards_after_first_shock_SHOCK_V2,rewards_got_shock_SHOCK_V2,start_time_SHOCK_V2,total_active_lever_presses_SHOCK_V2,total_inactive_lever_presses_SHOCK_V2,total_reward_SHOCK_V2,total_shocks_SHOCK_V2
0,933000320046919,11:25:09,"[15.0, 53.0, 100.0, 433.0, 588.0, 760.0, 972.0...",12.0,"[8.0, 9.0, 10.0, 13.0, 15.0, 18.0]",10:23:32,23.0,0.0,20.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000320047277,11:25:09,"[30.0, 714.0, 804.0, 927.0, 1094.0, 1229.0, 13...",11.0,"[4.0, 5.0, 9.0, 11.0]",10:24:23,15.0,31.0,15.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000320047132,13:05:55,"[25.0, 148.0, 257.0, 360.0, 399.0, 480.0, 747....",4.0,"[6.0, 8.0, 9.0]",12:05:13,10.0,4.0,10.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000320047119,14:30:00,"[13.0, 63.0, 85.0, 126.0, 168.0, 193.0, 218.0,...",26.0,"[1.0, 7.0, 10.0, 16.0, 17.0, 19.0, 23.0, 27.0]",13:29:23,43.0,1.0,27.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000320047312,10:02:06,"[140.0, 193.0, 230.0, 261.0, 309.0, 420.0, 593...",39.0,"[5.0, 7.0, 9.0, 11.0, 18.0, 20.0, 24.0, 28.0, ...",09:01:23,52.0,0.0,44.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
shock_unstacked = shock_unstacked[['rfid']+sorted(shock_unstacked.columns[1:])]
shock_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/shock_unstacked.csv')

## trial_pr

In [56]:
pr = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_pr.csv')
pr.head()

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,breakpoint,last_ratio,ratios,active_lever_presses,inactive_lever_presses,reward_presses
0,933000320188113,M1553,BSB273D,15,PR01,cocaine,1.0,08:37:54,10:30:22,2021-01-12,2021-01-12,62.0,NaN,1 2 4 6 9 12 15 20 25 32 40 50 62,352.0,21.0,13.0
1,933000320047476,M871,BSB273E,8,PR01,cocaine,3.0,09:11:54,11:48:59,2019-07-10,2019-07-10,6.0,NaN,1 2 4 6,13.0,1.0,4.0
2,933000320187081,M1171,BSB273E,11,PR03,cocaine,3.0,08:44:15,10:49:28,2020-05-12,2020-05-12,2.0,NaN,1 2,5.0,0.0,2.0
3,933000320188354,M1679,BSB273E,16,PR03,cocaine,11.0,08:42:20,10:25:24,2021-04-20,2021-04-20,4.0,NaN,1 2 4,7.0,0.0,3.0
4,933000120138642,F310,NaN,3,PR01,cocaine,10.0,07:45:28,10:42:18,2018-02-20,2018-02-20,40.0,NaN,1 2 4 6 9 12 15 20 25 32 40,179.0,2.0,11.0


In [57]:
pr['ratios'] = pr['ratios'].apply(deserialize_data)

In [58]:
pr_trimmed = pr.drop(columns=['subject', 'room', 'cohort', 'box','start_date','end_date'])
print(pr_trimmed.shape)
pr_trimmed.head()

(6983, 11)


,rfid,trial_id,drug,start_time,end_time,breakpoint,last_ratio,ratios,active_lever_presses,inactive_lever_presses,reward_presses
0,933000320188113,PR01,cocaine,08:37:54,10:30:22,62.0,NaN,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0, 15.0, 20.0, 25...",352.0,21.0,13.0
1,933000320047476,PR01,cocaine,09:11:54,11:48:59,6.0,NaN,"[1.0, 2.0, 4.0, 6.0]",13.0,1.0,4.0
2,933000320187081,PR03,cocaine,08:44:15,10:49:28,2.0,NaN,"[1.0, 2.0]",5.0,0.0,2.0
3,933000320188354,PR03,cocaine,08:42:20,10:25:24,4.0,NaN,"[1.0, 2.0, 4.0]",7.0,0.0,3.0
4,933000120138642,PR01,cocaine,07:45:28,10:42:18,40.0,NaN,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0, 15.0, 20.0, 25...",179.0,2.0,11.0


In [59]:
pr_keep = pd.merge(pr_trimmed, note_filter, how='left', left_on=['rfid','trial_id','drug'], right_on=['rfid','trial_id','drug'])
pr_keep = pr_keep[~(pr_keep['to_do']=='discard')]
pr_keep.drop(columns=['drug','to_do'],inplace=True)
pr_keep.shape

(6907, 10)

In [60]:
rows = unstack_data(pr_keep)
pr_unstacked = pd.concat(rows,ignore_index=True)
pr_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in pr_unstacked.columns[1:]]
print(pr_unstacked.shape)
pr_unstacked.head()

(1874, 57)


,rfid,active_lever_presses_PR01,breakpoint_PR01,end_time_PR01,inactive_lever_presses_PR01,last_ratio_PR01,ratios_PR01,reward_presses_PR01,start_time_PR01,active_lever_presses_PR02,...,reward_presses_TREATMENT03,start_time_TREATMENT03,active_lever_presses_TREATMENT04,breakpoint_TREATMENT04,end_time_TREATMENT04,inactive_lever_presses_TREATMENT04,last_ratio_TREATMENT04,ratios_TREATMENT04,reward_presses_TREATMENT04,start_time_TREATMENT04
0,933000320188113,352.0,62.0,10:30:22,21.0,NaN,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0, 15.0, 20.0, 25...",13.0,08:37:54,427.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000320047476,13.0,6.0,11:48:59,1.0,NaN,"[1.0, 2.0, 4.0, 6.0]",4.0,09:11:54,90.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000320187081,84.0,20.0,12:49:11,10.0,NaN,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0, 15.0, 20.0]",8.0,09:26:09,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000320188354,0.0,0.0,09:52:39,0.0,NaN,[],0.0,08:20:50,13.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000120138642,179.0,40.0,10:42:18,2.0,NaN,"[1.0, 2.0, 4.0, 6.0, 9.0, 12.0, 15.0, 20.0, 25...",11.0,07:45:28,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
pr_unstacked = pr_unstacked[['rfid']+sorted(pr_unstacked.columns[1:])]
pr_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/pr_unstacked.csv')

## trial_sha

In [62]:
sha = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_sha.csv')
sha['timeout_presses'] = sha['active_lever_presses'] - sha['reward_presses']
sha.head()

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000320188614,M1658,BSB273D,16,SHA07,cocaine,6.0,08:36:10,11:02:11,2021-03-15,2021-03-15,2.0,3.0,2.0,0.0,6909 7087,69 74 7044,6909 7087,NaN
1,933000320047019,M975,BSB273E,9,SHA07,cocaine,7.0,08:57:46,10:59:13,2019-10-01,2019-10-01,5.0,3.0,4.0,1.0,396 3634 6222 6236 6257,31 6126 6716,396 3634 6222 6257,6236
2,933000320049155,M1472,BSB273E,14,SHA08,cocaine,4.0,08:18:25,11:18:54,2020-10-15,2020-10-15,24.0,1.0,20.0,4.0,46 47 77 659 760 1093 1244 1443 1446 1524 2569...,83,46 77 659 760 1093 1244 1443 1524 2569 2617 32...,47 1446 3295 6864
3,933000320188719,M1780,BSB273E,17,SHA09,cocaine,12.0,08:30:08,10:42:40,2021-06-16,2021-06-16,3.0,0.0,3.0,0.0,219 616 4898,NaN,219 616 4898,NaN
4,933000320047169,F911,BSB273B,9,SHA02,cocaine,11.0,07:55:29,10:50:16,2019-09-24,2019-09-24,1.0,1.0,1.0,0.0,4807,4760,4807,NaN


In [63]:
sha['active_timestamps'] = sha['active_timestamps'].apply(deserialize_data)
sha['inactive_timestamps'] = sha['inactive_timestamps'].apply(deserialize_data)
sha['reward_timestamps'] = sha['reward_timestamps'].apply(deserialize_data)
sha['timeout_timestamps'] = sha['timeout_timestamps'].apply(deserialize_data)

In [64]:
sha_trimmed = sha.drop(columns=['subject', 'room', 'cohort', 'box','start_date','end_date'])
print(sha_trimmed.shape)
sha_trimmed.head()

(13499, 13)


,rfid,trial_id,drug,start_time,end_time,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000320188614,SHA07,cocaine,08:36:10,11:02:11,2.0,3.0,2.0,0.0,"[6909.0, 7087.0]","[69.0, 74.0, 7044.0]","[6909.0, 7087.0]",[]
1,933000320047019,SHA07,cocaine,08:57:46,10:59:13,5.0,3.0,4.0,1.0,"[396.0, 3634.0, 6222.0, 6236.0, 6257.0]","[31.0, 6126.0, 6716.0]","[396.0, 3634.0, 6222.0, 6257.0]",[6236.0]
2,933000320049155,SHA08,cocaine,08:18:25,11:18:54,24.0,1.0,20.0,4.0,"[46.0, 47.0, 77.0, 659.0, 760.0, 1093.0, 1244....",[83.0],"[46.0, 77.0, 659.0, 760.0, 1093.0, 1244.0, 144...","[47.0, 1446.0, 3295.0, 6864.0]"
3,933000320188719,SHA09,cocaine,08:30:08,10:42:40,3.0,0.0,3.0,0.0,"[219.0, 616.0, 4898.0]",[],"[219.0, 616.0, 4898.0]",[]
4,933000320047169,SHA02,cocaine,07:55:29,10:50:16,1.0,1.0,1.0,0.0,[4807.0],[4760.0],[4807.0],[]


In [65]:
sha_keep = pd.merge(sha_trimmed, note_filter, how='left', left_on=['rfid','trial_id','drug'], right_on=['rfid','trial_id','drug'])
sha_keep = sha_keep[~(sha_keep['to_do']=='discard')]
sha_keep.drop(columns=['drug','to_do'],inplace=True)
sha_keep.shape

(13468, 12)

In [66]:
rows = unstack_data(sha_keep)
sha_unstacked = pd.concat(rows,ignore_index=True)
sha_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in sha_unstacked.columns[1:]]
print(sha_unstacked.shape)
sha_unstacked.head()

(1848, 111)


,rfid,active_lever_presses_SHA01,active_timestamps_SHA01,end_time_SHA01,inactive_lever_presses_SHA01,inactive_timestamps_SHA01,reward_presses_SHA01,reward_timestamps_SHA01,start_time_SHA01,timeout_presses_SHA01,...,active_lever_presses_SHA11,active_timestamps_SHA11,end_time_SHA11,inactive_lever_presses_SHA11,inactive_timestamps_SHA11,reward_presses_SHA11,reward_timestamps_SHA11,start_time_SHA11,timeout_presses_SHA11,timeout_timestamps_SHA11
0,933000320188614,2.0,"[855.0, 860.0]",10:59:12,0.0,[],1.0,[855.0],08:58:31,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000320047019,1.0,[6727.0],10:57:01,1.0,[5537.0],1.0,[6727.0],08:50:17,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000320049155,4.0,"[579.0, 581.0, 582.0, 585.0]",11:44:53,2.0,"[2992.0, 3016.0]",1.0,[579.0],09:44:25,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000320188719,0.0,[],11:27:06,1.0,[1090.0],0.0,[],08:35:03,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000320047169,2.0,"[1470.0, 1627.0]",11:03:15,1.0,[1393.0],2.0,"[1470.0, 1627.0]",08:11:04,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
sha_unstacked = sha_unstacked[['rfid']+sorted(sha_unstacked.columns[1:])]
sha_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/sha_unstacked.csv')

## trial_lga

In [68]:
lga = pd.read_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Raw/trial_lga.csv')
lga['timeout_presses'] = lga['active_lever_presses'] - lga['reward_presses']
lga.head()

,rfid,subject,room,cohort,trial_id,drug,box,start_time,end_time,start_date,end_date,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000320186895,F1124,BSB273C,11,LGA14,cocaine,8.0,07:37:46,15:21:21,2020-05-01,2020-05-01,171.0,6.0,147.0,24.0,3 4 23 84 108 139 168 192 219 329 362 411 437 ...,162 164 164 165 166 519,3 23 84 108 139 168 192 219 329 362 411 437 45...,4 897 1322 1339 1344 1344 1355 1419 1479 3219 ...
1,933000320047476,M871,BSB273E,8,LGA07,cocaine,3.0,09:06:46,15:13:36,2019-07-19,2019-07-19,83.0,0.0,83.0,0.0,45 79 165 269 4039 4411 4439 4530 4775 4988 50...,NaN,45 79 165 269 4039 4411 4439 4530 4775 4988 50...,NaN
2,933000120138582,F406,NaN,4,LGA07,cocaine,6.0,07:43:27,13:59:32,2018-05-09,2018-05-09,126.0,0.0,118.0,8.0,33 40 587 638 666 696 728 779 817 856 977 1163...,NaN,33 587 638 666 696 728 779 817 856 977 1163 13...,40 2709 4027 15493 16252 17460 19377 20917
3,933000320525441,M2066,MTF134D,20,LGA04,cocaine,14.0,08:09:31,15:08:45,2022-03-24,2022-03-24,3.0,0.0,2.0,1.0,47 48 90,NaN,47 90,48
4,933000320047266,F414,BSB273B,4,LGA13,oxycodone,14.0,17:22:43,05:54:17,2019-07-22,2019-07-23,543.0,29.0,149.0,394.0,16 24 30 65 103 105 107 109 111 122 125 131 14...,56 2686 5790 6009 6907 7037 7037 7367 14234 14...,16 65 103 125 149 171 193 242 263 284 307 350 ...,24 30 105 107 109 111 122 131 143 143 150 151 ...


In [69]:
lga['active_timestamps'] = lga['active_timestamps'].apply(deserialize_data)
lga['inactive_timestamps'] = lga['inactive_timestamps'].apply(deserialize_data)
lga['reward_timestamps'] = lga['reward_timestamps'].apply(deserialize_data)
lga['timeout_timestamps'] = lga['timeout_timestamps'].apply(deserialize_data)

In [70]:
lga_trimmed = lga.drop(columns=['subject', 'room', 'cohort', 'box','start_date','end_date'])
print(lga_trimmed.shape)
lga_trimmed.head()

(33020, 13)


,rfid,trial_id,drug,start_time,end_time,active_lever_presses,inactive_lever_presses,reward_presses,timeout_presses,active_timestamps,inactive_timestamps,reward_timestamps,timeout_timestamps
0,933000320186895,LGA14,cocaine,07:37:46,15:21:21,171.0,6.0,147.0,24.0,"[3.0, 4.0, 23.0, 84.0, 108.0, 139.0, 168.0, 19...","[162.0, 164.0, 164.0, 165.0, 166.0, 519.0]","[3.0, 23.0, 84.0, 108.0, 139.0, 168.0, 192.0, ...","[4.0, 897.0, 1322.0, 1339.0, 1344.0, 1344.0, 1..."
1,933000320047476,LGA07,cocaine,09:06:46,15:13:36,83.0,0.0,83.0,0.0,"[45.0, 79.0, 165.0, 269.0, 4039.0, 4411.0, 443...",[],"[45.0, 79.0, 165.0, 269.0, 4039.0, 4411.0, 443...",[]
2,933000120138582,LGA07,cocaine,07:43:27,13:59:32,126.0,0.0,118.0,8.0,"[33.0, 40.0, 587.0, 638.0, 666.0, 696.0, 728.0...",[],"[33.0, 587.0, 638.0, 666.0, 696.0, 728.0, 779....","[40.0, 2709.0, 4027.0, 15493.0, 16252.0, 17460..."
3,933000320525441,LGA04,cocaine,08:09:31,15:08:45,3.0,0.0,2.0,1.0,"[47.0, 48.0, 90.0]",[],"[47.0, 90.0]",[48.0]
4,933000320047266,LGA13,oxycodone,17:22:43,05:54:17,543.0,29.0,149.0,394.0,"[16.0, 24.0, 30.0, 65.0, 103.0, 105.0, 107.0, ...","[56.0, 2686.0, 5790.0, 6009.0, 6907.0, 7037.0,...","[16.0, 65.0, 103.0, 125.0, 149.0, 171.0, 193.0...","[24.0, 30.0, 105.0, 107.0, 109.0, 111.0, 122.0..."


In [71]:
lga_keep = pd.merge(lga_trimmed, note_filter, how='left', left_on=['rfid','trial_id','drug'], right_on=['rfid','trial_id','drug'])
lga_keep = lga_keep[~(lga_keep['to_do']=='discard')]
lga_keep.drop(columns=['drug','to_do'],inplace=True)
lga_keep.shape

(32453, 12)

In [72]:
rows = unstack_data(lga_keep)
lga_unstacked = pd.concat(rows,ignore_index=True)
lga_unstacked.columns = ['rfid'] + [i[0]+'_'+i[1] for i in lga_unstacked.columns[1:]]
print(lga_unstacked.shape)
lga_unstacked.head()

(1857, 281)


,rfid,active_lever_presses_LGA01,active_timestamps_LGA01,end_time_LGA01,inactive_lever_presses_LGA01,inactive_timestamps_LGA01,reward_presses_LGA01,reward_timestamps_LGA01,start_time_LGA01,timeout_presses_LGA01,...,active_lever_presses_LGA28,active_timestamps_LGA28,end_time_LGA28,inactive_lever_presses_LGA28,inactive_timestamps_LGA28,reward_presses_LGA28,reward_timestamps_LGA28,start_time_LGA28,timeout_presses_LGA28,timeout_timestamps_LGA28
0,933000320186895,93.0,"[15.0, 21.0, 23.0, 50.0, 55.0, 87.0, 668.0, 74...",15:28:44,8.0,"[1.0, 8.0, 26.0, 46.0, 733.0, 3766.0, 6353.0, ...",66.0,"[15.0, 50.0, 87.0, 668.0, 745.0, 1175.0, 1520....",12:39:28,27.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000320047476,74.0,"[109.0, 155.0, 321.0, 432.0, 2476.0, 2519.0, 3...",15:42:36,3.0,"[5107.0, 14268.0, 18986.0]",68.0,"[109.0, 155.0, 321.0, 432.0, 2476.0, 2519.0, 3...",09:42:05,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000120138582,126.0,"[49.0, 56.0, 116.0, 161.0, 204.0, 214.0, 228.0...",13:50:30,0.0,[],118.0,"[49.0, 116.0, 161.0, 204.0, 228.0, 566.0, 682....",07:49:42,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000320525441,7.0,"[56.0, 80.0, 81.0, 308.0, 322.0, 19138.0, 1948...",15:07:37,5.0,"[311.0, 315.0, 318.0, 19770.0, 19774.0]",5.0,"[56.0, 80.0, 308.0, 19138.0, 19488.0]",08:09:00,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000320047266,74.0,"[2140.0, 2189.0, 2869.0, 2871.0, 2916.0, 3419....",05:55:26,25.0,"[2177.0, 2179.0, 2579.0, 2927.0, 3415.0, 3417....",65.0,"[2140.0, 2189.0, 2869.0, 2916.0, 3419.0, 4401....",17:11:00,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
lga_unstacked = lga_unstacked[['rfid']+sorted(lga_unstacked.columns[1:])]
lga_unstacked.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/lga_unstacked.csv')

# JOINING ALL TABLES HERE 

#### (9 tables in total, Notes table excluded since it has been merged to each of the trial data tables)

In [74]:
sub.shape

(2381, 105)

In [75]:
mmt_unstacked.shape

(2381, 37)

In [76]:
irr_trimmed.shape

(662, 34)

In [77]:
ti_trimmed.shape

(976, 5)

In [78]:
vf_trimmed.shape

(1066, 38)

In [79]:
shock_unstacked.shape

(891, 37)

In [80]:
pr_unstacked.shape

(1874, 57)

In [81]:
sha_unstacked.shape

(1848, 111)

In [82]:
lga_unstacked.shape

(1857, 281)

In [83]:
test = (
    sub.merge(mmt_unstacked, how='left', on='rfid')
        .merge(irr_trimmed, how='left', on='rfid')
        .merge(ti_trimmed, how='left', on='rfid')
        .merge(vf_trimmed, how='left', on='rfid')
        .merge(shock_unstacked, how='left', on='rfid')
        .merge(pr_unstacked, how='left', on='rfid')
        .merge(sha_unstacked, how='left', on='rfid')
        .merge(lga_unstacked, how='left', on='rfid')
)
test

,rfid,rat,cohort,experiment_group,drug_group,sex,arrival_date,age_at_arrival,uv,brevital,...,timeout_timestamps_LGA19,timeout_timestamps_LGA20,timeout_timestamps_LGA21,timeout_timestamps_LGA22,timeout_timestamps_LGA23,timeout_timestamps_LGA24,timeout_timestamps_LGA25,timeout_timestamps_LGA26,timeout_timestamps_LGA27,timeout_timestamps_LGA28
0,933000120124701,F1,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,933000120124703,F2,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,933000120117342,F3,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,933000120117333,F4,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,933000120117347,F5,1,Drug,Cocaine,Female,2017-06-29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2376,933000320625139,M1883,18,Naive,Oxycodone,Male,2022-08-31,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2377,933000320624877,M1884,18,Naive,Oxycodone,Male,2022-08-31,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2378,933000320625141,M1885,18,Naive,Oxycodone,Male,2022-08-31,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2379,933000320624891,M1886,18,Naive,Oxycodone,Male,2022-08-31,36.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
test.shape

(2381, 697)

In [85]:
test.columns.tolist()

['rfid',
 'rat',
 'cohort',
 'experiment_group',
 'drug_group',
 'sex',
 'arrival_date',
 'age_at_arrival',
 'uv',
 'brevital',
 'brevital_date',
 'brevital_technicians',
 'lga_15_date',
 'lga_16_date',
 'lga_17_date',
 'lga_18_date',
 'lga_19_date',
 'lga_20_date',
 'age_at_lga',
 'long_access_start_date',
 'long_access_end_date',
 'age_at_sha',
 'short_access_start_date',
 'short_access_end_date',
 'pre_shock_date',
 'shock_1_date',
 'shock_2_date',
 'shock_3_date',
 'female_swab_1_technicians',
 'female_swab_1_date',
 'female_swab_1_analysis',
 'female_swab_2_technicians',
 'female_swab_2_date',
 'female_swab_2_analysis',
 'female_swab_3_technicians',
 'female_swab_3_date',
 'female_swab_3_analysis',
 'irritability_1_technicians',
 'irritability_1_date',
 'irritability_2_technicians',
 'irritability_2_date',
 'von_frey_1_technicians',
 'von_frey_1_date',
 'von_frey_2_technicians',
 'von_frey_2_date',
 'tail_immersion_1_technicians',
 'tail_immersion_1_date',
 'tail_immersion_2_techn

In [95]:
ordered_cols = [
    'rfid',
    'rat',
    'cohort',
    'sex',
    'drug_group',
    'experiment_group',
    'dissection_group',
    'brevital',
    'last_good_session',
    'exit_code',
    'complete',
    'tissue_collected', 
    'exit_notes',
    'replaced_by',
    'coat_color',
    'ear_punch',
    'litter_number',
    'litter_size',
    'rack',
    'shipping_box',
    'uv',
    'female_swab_1_analysis',
    'female_swab_2_analysis',
    'female_swab_3_analysis',
    'reward_presses_SHA01',
    'reward_presses_SHA02',
    'reward_presses_SHA03',
    'reward_presses_SHA04',
    'reward_presses_SHA05',
    'reward_presses_SHA06',
    'reward_presses_SHA07',
    'reward_presses_SHA08',
    'reward_presses_SHA09',
    'reward_presses_SHA10',
    'active_lever_presses_SHA01',
    'active_lever_presses_SHA02',
    'active_lever_presses_SHA03',
    'active_lever_presses_SHA04',
    'active_lever_presses_SHA05',
    'active_lever_presses_SHA06',
    'active_lever_presses_SHA07',
    'active_lever_presses_SHA08',
    'active_lever_presses_SHA09',
    'active_lever_presses_SHA10',
    'timeout_presses_SHA01',
    'timeout_presses_SHA02', 
    'timeout_presses_SHA03',
    'timeout_presses_SHA04',
    'timeout_presses_SHA05',
    'timeout_presses_SHA06',
    'timeout_presses_SHA07',
    'timeout_presses_SHA08',
    'timeout_presses_SHA09',
    'timeout_presses_SHA10',
    'inactive_lever_presses_SHA01',
    'inactive_lever_presses_SHA02',
    'inactive_lever_presses_SHA03',
    'inactive_lever_presses_SHA04',
    'inactive_lever_presses_SHA05',
    'inactive_lever_presses_SHA06',
    'inactive_lever_presses_SHA07',
    'inactive_lever_presses_SHA08',
    'inactive_lever_presses_SHA09',
    'inactive_lever_presses_SHA10',
    'reward_presses_LGA01',
    'reward_presses_LGA02',
    'reward_presses_LGA03',
    'reward_presses_LGA04',
    'reward_presses_LGA05',
    'reward_presses_LGA06',
    'reward_presses_LGA07',
    'reward_presses_LGA08',
    'reward_presses_LGA09',
    'reward_presses_LGA10',
    'reward_presses_LGA11',
    'reward_presses_LGA12',
    'reward_presses_LGA13',
    'reward_presses_LGA14',
    'active_lever_presses_LGA01',
    'active_lever_presses_LGA02',
    'active_lever_presses_LGA03',
    'active_lever_presses_LGA04',
    'active_lever_presses_LGA05',
    'active_lever_presses_LGA06',
    'active_lever_presses_LGA07',
    'active_lever_presses_LGA08',
    'active_lever_presses_LGA09',
    'active_lever_presses_LGA10',
    'active_lever_presses_LGA11',
    'active_lever_presses_LGA12',
    'active_lever_presses_LGA13',
    'active_lever_presses_LGA14',
    'timeout_presses_LGA01',
    'timeout_presses_LGA02',
    'timeout_presses_LGA03',
    'timeout_presses_LGA04',
    'timeout_presses_LGA05',
    'timeout_presses_LGA06',
    'timeout_presses_LGA07',
    'timeout_presses_LGA08',
    'timeout_presses_LGA09',
    'timeout_presses_LGA10',
    'timeout_presses_LGA11',
    'timeout_presses_LGA12',
    'timeout_presses_LGA13',
    'timeout_presses_LGA14',
    'inactive_lever_presses_LGA01',
    'inactive_lever_presses_LGA02',
    'inactive_lever_presses_LGA03',
    'inactive_lever_presses_LGA04',
    'inactive_lever_presses_LGA05',
    'inactive_lever_presses_LGA06',
    'inactive_lever_presses_LGA07',
    'inactive_lever_presses_LGA08',
    'inactive_lever_presses_LGA09',
    'inactive_lever_presses_LGA10',
    'inactive_lever_presses_LGA11',
    'inactive_lever_presses_LGA12',
    'inactive_lever_presses_LGA13',
    'inactive_lever_presses_LGA14',
    'reward_presses_PR01',
    'reward_presses_PR02',
    'reward_presses_PR03',
    'active_lever_presses_PR01',
    'active_lever_presses_PR02',
    'active_lever_presses_PR03',
    'inactive_lever_presses_PR01',
    'inactive_lever_presses_PR02',
    'inactive_lever_presses_PR03',
    'breakpoint_PR01',
    'breakpoint_PR02',
    'breakpoint_PR03',
    'last_ratio_PR01',
    'last_ratio_PR02',
    'last_ratio_PR03',
    'treatment_1_group',
    'treatment_2_group',
    'treatment_3_group',
    'treatment_4_group',
    'reward_presses_TREATMENT01',
    'reward_presses_TREATMENT02',
    'reward_presses_TREATMENT03',
    'reward_presses_TREATMENT04',
    'active_lever_presses_TREATMENT01',
    'active_lever_presses_TREATMENT02',
    'active_lever_presses_TREATMENT03',
    'active_lever_presses_TREATMENT04',
    'inactive_lever_presses_TREATMENT01',
    'inactive_lever_presses_TREATMENT02',
    'inactive_lever_presses_TREATMENT03',
    'inactive_lever_presses_TREATMENT04',
    'breakpoint_TREATMENT01',
    'breakpoint_TREATMENT02',
    'breakpoint_TREATMENT03',
    'breakpoint_TREATMENT04',
    'last_ratio_TREATMENT01',
    'last_ratio_TREATMENT02',
    'last_ratio_TREATMENT03',
    'last_ratio_TREATMENT04',
    'total_reward_PRESHOCK',
    'total_reward_SHOCK_V1',
    'total_reward_SHOCK_V2',
    'total_reward_SHOCK_V3',
    'total_active_lever_presses_PRESHOCK',
    'total_active_lever_presses_SHOCK_V1',
    'total_active_lever_presses_SHOCK_V2',
    'total_active_lever_presses_SHOCK_V3',
    'total_inactive_lever_presses_PRESHOCK',
    'total_inactive_lever_presses_SHOCK_V1',
    'total_inactive_lever_presses_SHOCK_V2',
    'total_inactive_lever_presses_SHOCK_V3',
    'total_shocks_PRESHOCK',
    'total_shocks_SHOCK_V1',
    'total_shocks_SHOCK_V2',
    'total_shocks_SHOCK_V3',
    'rewards_after_first_shock_PRESHOCK',
    'rewards_after_first_shock_SHOCK_V1',
    'rewards_after_first_shock_SHOCK_V2',
    'rewards_after_first_shock_SHOCK_V3',
    'reward_presses_SHA11',
    'active_lever_presses_SHA11',
    'timeout_presses_SHA11',
    'inactive_lever_presses_SHA11',
    'reward_presses_LGA15',
    'reward_presses_LGA16',
    'reward_presses_LGA17',
    'reward_presses_LGA18',
    'reward_presses_LGA19',
    'reward_presses_LGA20',
    'reward_presses_LGA21',
    'reward_presses_LGA22',
    'reward_presses_LGA23',
    'reward_presses_LGA24',
    'reward_presses_LGA25',
    'reward_presses_LGA26',
    'reward_presses_LGA27',
    'reward_presses_LGA28',
    'active_lever_presses_LGA15',
    'active_lever_presses_LGA16',
    'active_lever_presses_LGA17',
    'active_lever_presses_LGA18',
    'active_lever_presses_LGA19',
    'active_lever_presses_LGA20',
    'active_lever_presses_LGA21',
    'active_lever_presses_LGA22',
    'active_lever_presses_LGA23',
    'active_lever_presses_LGA24',
    'active_lever_presses_LGA25',
    'active_lever_presses_LGA26',
    'active_lever_presses_LGA27',
    'active_lever_presses_LGA28',
    'timeout_presses_LGA15',
    'timeout_presses_LGA16',
    'timeout_presses_LGA17',
    'timeout_presses_LGA18',
    'timeout_presses_LGA19',
    'timeout_presses_LGA20',
    'timeout_presses_LGA21',
    'timeout_presses_LGA22',
    'timeout_presses_LGA23',
    'timeout_presses_LGA24',
    'timeout_presses_LGA25',
    'timeout_presses_LGA26',
    'timeout_presses_LGA27',
    'timeout_presses_LGA28',
    'inactive_lever_presses_LGA15',
    'inactive_lever_presses_LGA16',
    'inactive_lever_presses_LGA17',
    'inactive_lever_presses_LGA18',
    'inactive_lever_presses_LGA19',
    'inactive_lever_presses_LGA20',
    'inactive_lever_presses_LGA21',
    'inactive_lever_presses_LGA22',
    'inactive_lever_presses_LGA23',
    'inactive_lever_presses_LGA24',
    'inactive_lever_presses_LGA25',
    'inactive_lever_presses_LGA26',
    'inactive_lever_presses_LGA27',
    'inactive_lever_presses_LGA28',
    'von_frey_1_force',
    'von_frey_1_time',
    'von_frey_2_force',
    'von_frey_2_time',
    'von_frey_difference_force',
    'tail_immersion_1_time',
    'tail_immersion_2_time',
    'tail_immersion_3_time',
    'tail_immersion_difference_tolerance',
    'def_bsl_ave',
    'agg_bsl_ave',
    'total_bsl_ave',
    'def_drug_ave',
    'agg_drug_ave',
    'total_drug_ave',
    'diff_ave_def',
    'diff_ave_agg',
    'diff_ave_total',
    'reward_timestamps_SHA01',
    'reward_timestamps_SHA02',
    'reward_timestamps_SHA03',
    'reward_timestamps_SHA04',
    'reward_timestamps_SHA05',
    'reward_timestamps_SHA06',
    'reward_timestamps_SHA07',
    'reward_timestamps_SHA08',
    'reward_timestamps_SHA09',
    'reward_timestamps_SHA10',
    'active_timestamps_SHA01',
    'active_timestamps_SHA02',
    'active_timestamps_SHA03',
    'active_timestamps_SHA04',
    'active_timestamps_SHA05',
    'active_timestamps_SHA06',
    'active_timestamps_SHA07',
    'active_timestamps_SHA08',
    'active_timestamps_SHA09',
    'active_timestamps_SHA10',
    'timeout_timestamps_SHA01',
    'timeout_timestamps_SHA02',
    'timeout_timestamps_SHA03',
    'timeout_timestamps_SHA04',
    'timeout_timestamps_SHA05',
    'timeout_timestamps_SHA06',
    'timeout_timestamps_SHA07',
    'timeout_timestamps_SHA08',
    'timeout_timestamps_SHA09',
    'timeout_timestamps_SHA10',
    'inactive_timestamps_SHA01',
    'inactive_timestamps_SHA02',
    'inactive_timestamps_SHA03',
    'inactive_timestamps_SHA04',
    'inactive_timestamps_SHA05',
    'inactive_timestamps_SHA06',
    'inactive_timestamps_SHA07',
    'inactive_timestamps_SHA08',
    'inactive_timestamps_SHA09',
    'inactive_timestamps_SHA10',
    'reward_timestamps_LGA01',
    'reward_timestamps_LGA02',
    'reward_timestamps_LGA03',
    'reward_timestamps_LGA04',
    'reward_timestamps_LGA05',
    'reward_timestamps_LGA06',
    'reward_timestamps_LGA07',
    'reward_timestamps_LGA08',
    'reward_timestamps_LGA09',
    'reward_timestamps_LGA10',
    'reward_timestamps_LGA11',
    'reward_timestamps_LGA12',
    'reward_timestamps_LGA13',
    'reward_timestamps_LGA14',
    'active_timestamps_LGA01',
    'active_timestamps_LGA02',
    'active_timestamps_LGA03',
    'active_timestamps_LGA04',
    'active_timestamps_LGA05',
    'active_timestamps_LGA06',
    'active_timestamps_LGA07',
    'active_timestamps_LGA08',
    'active_timestamps_LGA09',
    'active_timestamps_LGA10',
    'active_timestamps_LGA11',
    'active_timestamps_LGA12',
    'active_timestamps_LGA13',
    'active_timestamps_LGA14',
    'timeout_timestamps_LGA01',
    'timeout_timestamps_LGA02',
    'timeout_timestamps_LGA03',
    'timeout_timestamps_LGA04',
    'timeout_timestamps_LGA05',
    'timeout_timestamps_LGA06',
    'timeout_timestamps_LGA07',
    'timeout_timestamps_LGA08',
    'timeout_timestamps_LGA09',
    'timeout_timestamps_LGA10',
    'timeout_timestamps_LGA11',
    'timeout_timestamps_LGA12',
    'timeout_timestamps_LGA13',
    'timeout_timestamps_LGA14',
    'inactive_timestamps_LGA01',
    'inactive_timestamps_LGA02',
    'inactive_timestamps_LGA03',
    'inactive_timestamps_LGA04',
    'inactive_timestamps_LGA05',
    'inactive_timestamps_LGA06',
    'inactive_timestamps_LGA07',
    'inactive_timestamps_LGA08',
    'inactive_timestamps_LGA09',
    'inactive_timestamps_LGA10',
    'inactive_timestamps_LGA11',
    'inactive_timestamps_LGA12',
    'inactive_timestamps_LGA13',
    'inactive_timestamps_LGA14',
    'ratios_PR01',
    'ratios_PR02',
    'ratios_PR03',
    'ratios_TREATMENT01',
    'ratios_TREATMENT02',
    'ratios_TREATMENT03',
    'ratios_TREATMENT04',
    'group_pre_shock',
    'group_shock',
    'rewards_got_shock_PRESHOCK',
    'rewards_got_shock_SHOCK_V1',
    'rewards_got_shock_SHOCK_V2',
    'rewards_got_shock_SHOCK_V3',
    'reward_timestamps_PRESHOCK',
    'reward_timestamps_SHOCK_V1',
    'reward_timestamps_SHOCK_V2',
    'reward_timestamps_SHOCK_V3',
    'reward_timestamps_SHA11',
    'active_timestamps_SHA11',
    'timeout_timestamps_SHA11',
    'inactive_timestamps_SHA11',
    'reward_timestamps_LGA15',
    'reward_timestamps_LGA16',
    'reward_timestamps_LGA17',
    'reward_timestamps_LGA18',
    'reward_timestamps_LGA19',
    'reward_timestamps_LGA20',
    'reward_timestamps_LGA21',
    'reward_timestamps_LGA22',
    'reward_timestamps_LGA23',
    'reward_timestamps_LGA24',
    'reward_timestamps_LGA25',
    'reward_timestamps_LGA26',
    'reward_timestamps_LGA27',
    'reward_timestamps_LGA28',
    'active_timestamps_LGA15',
    'active_timestamps_LGA16',
    'active_timestamps_LGA17',
    'active_timestamps_LGA18',
    'active_timestamps_LGA19',
    'active_timestamps_LGA20',
    'active_timestamps_LGA21',
    'active_timestamps_LGA22',
    'active_timestamps_LGA23',
    'active_timestamps_LGA24',
    'active_timestamps_LGA25',
    'active_timestamps_LGA26',
    'active_timestamps_LGA27',
    'active_timestamps_LGA28',
    'timeout_timestamps_LGA15',
    'timeout_timestamps_LGA16',
    'timeout_timestamps_LGA17',
    'timeout_timestamps_LGA18',
    'timeout_timestamps_LGA19',
    'timeout_timestamps_LGA20',
    'timeout_timestamps_LGA21',
    'timeout_timestamps_LGA22',
    'timeout_timestamps_LGA23',
    'timeout_timestamps_LGA24',
    'timeout_timestamps_LGA25',
    'timeout_timestamps_LGA26',
    'timeout_timestamps_LGA27',
    'timeout_timestamps_LGA28',
    'inactive_timestamps_LGA15',
    'inactive_timestamps_LGA16',
    'inactive_timestamps_LGA17',
    'inactive_timestamps_LGA18',
    'inactive_timestamps_LGA19',
    'inactive_timestamps_LGA20',
    'inactive_timestamps_LGA21',
    'inactive_timestamps_LGA22',
    'inactive_timestamps_LGA23',
    'inactive_timestamps_LGA24',
    'inactive_timestamps_LGA25',
    'inactive_timestamps_LGA26',
    'inactive_timestamps_LGA27',
    'inactive_timestamps_LGA28',
    'vf1_right_force_1',
    'vf1_right_force_2',
    'vf1_right_force_3',
    'vf1_right_force_avg',
    'vf1_right_time_1',
    'vf1_right_time_2',
    'vf1_right_time_3',
    'vf1_right_avg',
    'vf1_left_force_1',
    'vf1_left_force_2',
    'vf1_left_force_3',
    'vf1_left_force_avg',
    'vf1_left_time_1',
    'vf1_left_time_2',
    'vf1_left_time_3',
    'vf1_left_avg',
    'vf2_right_force_1',
    'vf2_right_force_2',
    'vf2_right_force_3',
    'vf2_right_force_avg',
    'vf2_right_time_1',
    'vf2_right_time_2',
    'vf2_right_time_3',
    'vf2_right_avg',
    'vf2_left_force_1',
    'vf2_left_force_2',
    'vf2_left_force_3',
    'vf2_left_force_avg',
    'vf2_left_time_1',
    'vf2_left_time_2',
    'vf2_left_time_3',
    'vf2_left_avg',
    'def_bsl_irr_1',
    'agg_bsl_irr_1',
    'irr_bsl_tot_1',
    'def_bsl_irr_2',
    'agg_bsl_irr_2',
    'irr_bsl_total_2',
    'def_bsl_irr_3',
    'agg_bsl_irr_3',
    'irr_bsl_total_3',
    'def_drug_irr_1',
    'agg_drug_irr_1',
    'irr_drug_tot_1',
    'def_drug_irr_2',
    'agg_drug_irr_2',
    'irr_drug_total_2',
    'def_drug_irr_3',
    'agg_drug_irr_3',
    'irr_drug_total_3',
    'measurement_value_feces_1',
    'measurement_value_feces_2',
    'measurement_value_feces_3',
    'measurement_value_feces_4',
    'measurement_value_urine_1',
    'measurement_value_urine_2',
    'measurement_value_weight_1',
    'measurement_value_weight_2',
    'measurement_value_weight_3',
    'measurement_value_weight_4',
    'measurement_value_weight_5',
    'measurement_value_weight_6',
    'measurement_value_weight_7',
    'measurement_value_weight_8',
    'measurement_value_weight_9',
    'measurement_value_weight_10',
    'measurement_value_weight_11',
    'measurement_value_weight_12',
    'days_of_experiment',
    'age_at_arrival',
    'age_at_surgery',
    'age_at_sha',
    'age_at_lga',
    'age_at_dissection',
    'dissection_date',
    'date_of_death',
    'exit_day',
    'date_of_birth',
    'date_of_wean',
    'date_of_ship',
    'arrival_date',
    'date_of_eye_bleed',
    'surgery_date',
    'recatheter_surgery_date',
    'short_access_start_date',
    'short_access_end_date',
    'progressive_ratio_1_date',
    'long_access_start_date',
    'long_access_end_date',
    'progressive_ratio_2_date',
    'pre_shock_date',
    'shock_1_date',
    'shock_2_date',
    'shock_3_date',
    'progressive_ratio_3_date',
    'brevital_date',
    'start_time_SHA01',
    'end_time_SHA01',
    'start_time_SHA02',
    'end_time_SHA02',
    'start_time_SHA03',
    'end_time_SHA03',
    'start_time_SHA04',
    'end_time_SHA04',
    'start_time_SHA05',
    'end_time_SHA05',
    'start_time_SHA06',
    'end_time_SHA06',
    'start_time_SHA07',
    'end_time_SHA07',
    'start_time_SHA08',
    'end_time_SHA08',
    'start_time_SHA09',
    'end_time_SHA09',
    'start_time_SHA10',
    'end_time_SHA10',
    'start_time_LGA01',
    'end_time_LGA01',
    'start_time_LGA02',
    'end_time_LGA02',
    'start_time_LGA03',
    'end_time_LGA03',
    'start_time_LGA04',
    'end_time_LGA04',
    'start_time_LGA05',
    'end_time_LGA05',
    'start_time_LGA06',
    'end_time_LGA06',
    'start_time_LGA07',
    'end_time_LGA07',
    'start_time_LGA08',
    'end_time_LGA08',
    'start_time_LGA09',
    'end_time_LGA09',
    'start_time_LGA10',
    'end_time_LGA10',
    'start_time_LGA11',
    'end_time_LGA11',
    'start_time_LGA12',
    'end_time_LGA12',
    'start_time_LGA13',
    'end_time_LGA13',
    'start_time_LGA14',
    'end_time_LGA14',
    'start_time_PR01',
    'end_time_PR01',
    'start_time_PR02',
    'end_time_PR02',
    'start_time_PR03',
    'end_time_PR03',
    'lga_post_treatment_1_date',
    'lga_post_treatment_2_date',
    'lga_post_treatment_3_date',
    'lga_post_treatment_4_date',
    'treatment_1_start_time',
    'treatment_2_start_time',
    'treatment_3_start_time',
    'treatment_4_start_time',
    'start_time_TREATMENT01',
    'end_time_TREATMENT01', 
    'start_time_TREATMENT02',
    'end_time_TREATMENT02',
    'start_time_TREATMENT03',
    'end_time_TREATMENT03',
    'start_time_TREATMENT04',
    'end_time_TREATMENT04',
    'start_time_PRESHOCK',
    'end_time_PRESHOCK',
    'start_time_SHOCK_V1',
    'end_time_SHOCK_V1',
    'start_time_SHOCK_V2',
    'end_time_SHOCK_V2',
    'start_time_SHOCK_V3',
    'end_time_SHOCK_V3',
    'start_time_SHA11',
    'end_time_SHA11',
    'start_time_LGA15',
    'end_time_LGA15',
    'start_time_LGA16',
    'end_time_LGA16',
    'start_time_LGA17',
    'end_time_LGA17',
    'start_time_LGA18',
    'end_time_LGA18',
    'start_time_LGA19',
    'end_time_LGA19',
    'start_time_LGA20',
    'end_time_LGA20',
    'start_time_LGA21',
    'end_time_LGA21',
    'start_time_LGA22',
    'end_time_LGA22',
    'start_time_LGA23',
    'end_time_LGA23',
    'start_time_LGA24',
    'end_time_LGA24',
    'start_time_LGA25',
    'end_time_LGA25',
    'start_time_LGA26',
    'end_time_LGA26',
    'start_time_LGA27',
    'end_time_LGA27',
    'start_time_LGA28',
    'end_time_LGA28',
    'treatment_1_date',
    'treatment_2_date',
    'treatment_3_date',
    'treatment_4_date',
    'von_frey_1_date',
    'von_frey_2_date',
    'tail_immersion_1_date',
    'tail_immersion_2_date',
    'tail_immersion_3_date',
    'irritability_1_date',
    'irritability_2_date',
    'female_swab_1_date',
    'female_swab_2_date',
    'female_swab_3_date',
    'lga_pre_treatment_1_date',
    'lga_pre_treatment_2_date',
    'lga_pre_treatment_3_date',
    'lga_pre_treatment_4_date',
    'lga_15_date',
    'lga_16_date',
    'lga_17_date',
    'lga_18_date',
    'lga_19_date',
    'lga_20_date',
    'handled_by',
    'surgeon',
    'surgery_assist',
    'recatheter_surgeon',
    'brevital_technicians',
    'irr_bsl_scorer_1',
    'irr_bsl_scorer_2',
    'irr_bsl_scorer_3',
    'irr_drug_scorer_1',
    'irr_drug_scorer_2',
    'irr_drug_scorer_3',
    'irritability_1_technicians',
    'irritability_2_technicians',
    'von_frey_1_technicians',
    'von_frey_2_technicians',
    'tail_immersion_1_technicians',
    'tail_immersion_2_technicians',
    'tail_immersion_3_technicians',
    'female_swab_1_technicians',
    'female_swab_2_technicians',
    'female_swab_3_technicians',
    'technician_urine_1',
    'technician_urine_2',
    'technician_feces_1',
    'technician_feces_2',
    'technician_feces_3',
    'technician_feces_4',
    'technician_weight_1',
    'technician_weight_2',
    'technician_weight_3',
    'technician_weight_4',
    'technician_weight_5',
    'technician_weight_6',
    'technician_weight_7',
    'technician_weight_8',
    'technician_weight_9',
    'technician_weight_10',
    'technician_weight_11',
    'technician_weight_12',
]

In [96]:
len(ordered_cols)

697

In [100]:
test = test[ordered_cols]

In [101]:
test.to_csv('/Users/yunyihuang/George Lab Dropbox/George_Lab/Experiments/DataStream/DataStream_Database_01-07-2023/Unstacked/datastream_combined_tables_no_dates.csv')

In [ ]:
# "MTF134EC15HSOXYLGA" : [M1569 ......]